In [ ]:
from spacepy.pybats import bats
from scipy.constants import m_p
from astropy import units as u
import numpy as np

In [ ]:
BATS_FILE = '/home/dedasilv/data/3d__var_4_t00030600_n00022500.out'

In [ ]:
mhd = bats.IdlFile(BATS_FILE)

In [ ]:
mhd.keys()

# Coordinates

In [ ]:
x = mhd['x'] * u.R_earth
y = mhd['y'] * u.R_earth
z = mhd['z'] * u.R_earth

# Number Density

In [ ]:
n = mhd['rho'] / m_p / 1e27
n *= u.cm**(-3)

plt.title('number density')
plt.hist(n, bins=40)
plt.xlabel('$cm^{-3}$')
None

# Magnetic Field

In [ ]:
bx = mhd['bx'] * u.nT
by = mhd['by'] * u.nT
bz = mhd['bz'] * u.nT
btotal = np.sqrt(bx**2 + by**2 + bz**2)

In [ ]:
mask = (z < .1*u.R_earth) & (z > -.1*u.R_earth) & (x > 0 * u.R_earth)
from matplotlib.colors import LogNorm

plt.scatter(x.value[mask], y.value[mask], c=btotal.value[mask], norm=LogNorm())
plt.colorbar()

# Flow Velocity

In [ ]:
ux = mhd['ux'] * u.km/u.s
uy = mhd['uy'] * u.km/u.s
uz = mhd['uz'] * u.km/u.s
utotal = np.sqrt(ux**2 + uy**2 + uz**2)

In [ ]:
plt.hist(utotal, bins=40)
plt.xlabel('km/s')
plt.title('flow speed')
None

# Electric Field

In [ ]:
Ex, Ey, Ez = -np.cross([ux, uy, uz], [bx, by, bz], axis=0)
Ex.shape

In [ ]:
units = bx.unit * ux.unit
Ex *= units
Ey *= units
Ez *= units

better_units = u.mV/u.m
Ex = Ex.to(better_units)
Ey = Ey.to(better_units)
Ez = Ez.to(better_units)
Etotal = np.sqrt(Ex**2 + Ey**2 + Ez**2)

In [ ]:
plt.hist(Etotal, bins=40)
plt.title('Etotal')
plt.xlabel('mV/m')
None

# Pressure

In [ ]:
p = mhd['p'] * u.nPa
T = (p / n).to(u.eV)

In [ ]:
plt.hist(p, bins=40)
plt.title('Pressure')
plt.xlabel('nPa')
None

In [ ]:
plt.hist(T, bins=40)
plt.title('Temperature')
plt.xlabel('eV')
None

# Test Regrid

In [ ]:
import pyvista as pv
import vtk
import time

In [ ]:
xaxis = np.arange(-15, 15, .15)
yaxis = np.arange(-15, 15, .15)
zaxis = np.arange(-5, 5, .15)
X, Y, Z = np.meshgrid(xaxis, yaxis, zaxis)

In [ ]:
point_cloud = pv.PolyData(np.transpose([x, y, z]))
point_cloud['Bx'] = bx.flatten(order='F')
point_cloud['By'] = by.flatten(order='F')
point_cloud['Bz'] = bz.flatten(order='F')
point_cloud['Ex'] = Ex.flatten(order='F')
point_cloud['Ey'] = Ey.flatten(order='F')
point_cloud['Ez'] = Ez.flatten(order='F')
point_cloud['n'] = n.flatten(order='F')
point_cloud['T'] = T.flatten(order='F')

In [ ]:
start_time = time.time()

points_search = pv.PolyData(np.transpose([X.flatten(), Y.flatten(), Z.flatten()]))
interp = vtk.vtkPointInterpolator()  
interp.SetInputData(points_search)
interp.SetSourceData(point_cloud)
interp.GetKernel().SetRadius(0.1)
interp.Update()

interp_result = pv.PolyData(interp.GetOutput())

print('took', time.time() - start_time, 's')

In [ ]:
# Make MagneticFieldModel
x_grid = interp_result.points[:, 0].reshape(X.shape)
y_grid = interp_result.points[:, 1].reshape(X.shape)
z_grid = interp_result.points[:, 2].reshape(X.shape)
r_grid = np.sqrt(x_grid**2 + y_grid**2 + z_grid**2)

Bx_regrid = interp_result['Bx'].reshape(X.shape)
By_regrid = interp_result['By'].reshape(X.shape)
Bz_regrid = interp_result['Bz'].reshape(X.shape)

In [ ]:
Bz_regrid.shape

# Test Field Line Trace

In [ ]:
from dataclasses import dataclass

In [ ]:
class MagneticFieldModel:
    """Represents a magnetic field model, with methods for sampling the
    magnetic field at an aribtrary point.
    """
    def __init__(self, x, y, z, Bx, By, Bz, inner_boundary):
        self.x = x
        self.y = y
        self.z = z
        self.Bx = Bx
        self.By = By
        self.Bz = Bz
        self.inner_boundary = inner_boundary

        B = np.empty((Bx.size, 3))
        B[:, 0] = Bx.flatten(order="F")
        B[:, 1] = By.flatten(order="F")
        B[:, 2] = Bz.flatten(order="F")
        self._mesh = pv.StructuredGrid(x, y, z)
        self._mesh.point_data["B"] = B

    def trace_field_line(self, starting_point, step_size):
        """Perform a field line trace. Implements RK45 in both directions,
        stopping when outside the grid.

        Parameters
        ----------
        starting_point : tuple of floats
            Starting point of the field line trace, as (x, y, z) tuple of
            floats, in units of Re. Trace will go in both directions until it hits
            the model inner or outer boundary.
        step_size : float, optional
            Step size to use with the field line trace. If not sure, try 1e-3.

        Returns
        -------
        trace : :py:class:`FieldLineTrace`
            Coordinates and magnetic field vector along the field line trace
        """
        pv_trace = self._mesh.streamlines(
            "B",
            start_position=starting_point,
            terminal_speed=0.0,
            max_step_length=step_size * 10,
            min_step_length=step_size / 10,
            initial_step_length=step_size,
            step_unit="l",
            max_steps=1_000_000,
            interpolator_type="c",
        )

        return FieldLineTrace(points=pv_trace.points, B=pv_trace["B"])

@dataclass
class FieldLineTrace:
    """Class to hold the results of a field line trace.

    Parameters
    ----------
    points : array, shape (n, 3)
       Positions along field line trace, in SM coordinate system and units of Re
    B : array, shape (n, 3)
       Magnetic field vector along field line trace, in SM coordinates and units
       of Gauss
    """

    points: object
    B: object

In [ ]:
model = MagneticFieldModel(X, Y, Z, Bx_regrid, By_regrid, Bz_regrid, inner_boundary=2.5)

In [ ]:
trace = model.trace_field_line((5, 0, 0), 1e-3)

In [ ]:
I = np.argsort(trace.points[:, 2] / trace.points[:, 0])
plt.plot(trace.points[I, 0], trace.points[I, 2])